In [ ]:
!pip install yfinance pandas tensorflow

In [ ]:
!pip install numpy==1.25.2 --force-reinstall

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 59.7 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 1.25.2 which is incompatible.
blosc2 3.3.0 requires numpy>=1.26, but you have numpy 1.25.2 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.25.2 which is incompatible.


In [ ]:
!pip install pandas_ta --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 10.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218910 sha256=490855cc911649bee4dc022b8594096c3cb9fee69acad2e5269157e6ad590f3f
  Stored in directory: /root/.cache/pip/wheels/7f/33/8b/50b245c5c65433cd8f5cb24ac15d97e5a3db2d41a8b6ae957d
Successfully built pandas_ta


In [ ]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime, timedelta
import pandas_ta as pta

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

#import pygad

In [ ]:
# 종목 리스트 - 소비재 섹터 내 10개 대표 종목
tickers = ["HD", "MCD", "LOW", "TJX", "SBUX", "NKE", "MAR", "CMG", "ORLY", "TGT"]

# 결과를 담을 빈 리스트
all_dfs = []

In [ ]:
for ticker in tickers:
    # 2.1 기본 가격 데이터 다운로드
    df = yf.download(ticker, period="3y", interval="1d", auto_adjust=False)
    df = df[["Open","High","Low","Close","Adj Close","Volume"]].dropna()

    # 2.2 MultiIndex 평탄화 (Colab 등에서 가끔 생김)
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.get_level_values(0)

    # 2.3 30개 TA 지표 계산
    # — 추세 지표 —
    df.ta.sma(length=20, append=True); df.ta.sma(length=60, append=True)
    df.ta.ema(length=20, append=True); df.ta.ema(length=60, append=True)
    df.ta.wma(length=20, append=True); df.ta.wma(length=60, append=True)
    df.ta.hma(length=20, append=True); df.ta.hma(length=60, append=True)
    df.ta.macd(append=True)
    df.ta.psar(append=True)
    df.ta.tema(length=20, append=True)
    df.ta.adx(append=True)

    # — 모멘텀 지표 —
    df.ta.rsi(length=14, append=True)
    df.ta.stoch(append=True)
    df.ta.roc(length=10, append=True)
    df.ta.cci(length=20, append=True)
    df.ta.mom(length=10, append=True)
    df.ta.trix(length=15, append=True)
    df.ta.cmo(length=14, append=True)
    df.ta.tsi(append=True)

    # — 변동성 지표 —
    df.ta.atr(length=14, append=True)
    df.ta.bbands(length=20, append=True)
    df.ta.kc(append=True)
    df.ta.donchian(append=True)
    df.ta.natr(length=14, append=True)

    # — 거래량 지표 —
    df.ta.obv(append=True)
    df.ta.mfi(length=14, append=True)
    df.ta.ad(append=True)
    df.ta.efi(append=True)
    df.ta.cmf(length=20, append=True)

    # — 기타/종합 지표 —
    df.ta.vwap(append=True)
    df.ta.dpo(length=20, append=True)
    df.ta.pgo(append=True)
    df.ta.bop(append=True)

    # 2.4 Ticker 칼럼 추가
    df["Ticker"] = ticker

    # 2.5 리스트에 저장
    all_dfs.append(df)

# 3. 모든 종목 합치기
all_df = pd.concat(all_dfs)

# 4. CSV로 저장
all_df.to_csv("TA_data_30.csv", index=True)

print("✅ 저장 완료: TA_data_30_.csv — 총 로우:", len(all_df))

[*********************100%***********************]  1 of 1 completed
<ipython-input-18-a390454d8ffc>:40: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[1.03469461e+09 1.48074190e+09 1.03280200e+09 1.28539065e+09
 1.29228509e+09 1.01986513e+09 1.25195132e+09 1.54908177e+09
 1.28019632e+09 1.16353653e+09 1.25952843e+09 3.73932612e+09
 1.50303135e+09 1.21302788e+09 1.34499863e+09 1.25880305e+09
 1.19391367e+09 1.23653103e+09 9.02196607e+08 1.25620291e+09
 1.02544095e+09 2.88830204e+09 7.18576110e+08 1.10830744e+09
 7.83844361e+08 9.10254301e+08 6.25076366e+08 8.93276426e+08
 5.41059220e+08 7.04166549e+08 7.74832191e+08 8.10366521e+08
 7.75461969e+08 7.84583500e+08 7.78499306e+08 7.63625863e+08
 5.95473597e+08 7.34684739e+08 1.18969904e+09 9.70595051e+08
 1.00689660e+09 1.01522284e+09 8.35074373e+08 6.24942510e+08
 7.83436617e+08 1.20230294e+09 7.61654870e+08 7.12457333e+08
 1.22809965e+09 3.27326572e+09 1.0

✅ 저장 완료: TA_data_30_.csv — 총 로우: 7530


In [ ]:
# 1) CSV 로드
df = pd.read_csv("TA_data_30.csv", index_col=0)

# 2) 빈 signals DataFrame 생성 (날짜 인덱스만)
signals = pd.DataFrame(index=df.index)

# === 1. 추세(Trend) 지표 ===

# 1-1) SMA(20) 기울기
signals['SMA_signal'] = 0
signals.loc[df['SMA_20'] > df['SMA_20'].shift(1), 'SMA_signal'] = -1
signals.loc[df['SMA_20'] < df['SMA_20'].shift(1), 'SMA_signal'] = 1

# 1-2) EMA(20) 기울기
signals['EMA_signal'] = 0
signals.loc[df['EMA_20'] > df['EMA_20'].shift(1), 'EMA_signal'] = -1
signals.loc[df['EMA_20'] < df['EMA_20'].shift(1), 'EMA_signal'] = 1

# 1-3) WMA(20) 기울기
signals['WMA_signal'] = 0
signals.loc[df['WMA_20'] > df['WMA_20'].shift(1), 'WMA_signal'] = -1
signals.loc[df['WMA_20'] < df['WMA_20'].shift(1), 'WMA_signal'] = 1

# 1-4) HMA(20) 기울기
signals['HMA_signal'] = 0
signals.loc[df['HMA_20'] > df['HMA_20'].shift(1), 'HMA_signal'] = -1
signals.loc[df['HMA_20'] < df['HMA_20'].shift(1), 'HMA_signal'] = 1

# 1-5) TEMA(20) 기울기
signals['TEMA_signal'] = 0
signals.loc[df['TEMA_20'] > df['TEMA_20'].shift(1), 'TEMA_signal'] = -1
signals.loc[df['TEMA_20'] < df['TEMA_20'].shift(1), 'TEMA_signal'] = 1

# 1-6) MACD 교차
signals['MACD_signal'] = 0
signals.loc[df['MACD_12_26_9'] > df['MACDs_12_26_9'], 'MACD_signal'] = -1
signals.loc[df['MACD_12_26_9'] < df['MACDs_12_26_9'], 'MACD_signal'] = 1

# 1-7) PSAR 반전
signals['PSAR_signal'] = 0
signals.loc[df['Close'] > df['PSARs_0.02_0.2'], 'PSAR_signal'] = -1
signals.loc[df['Close'] < df['PSARl_0.02_0.2'], 'PSAR_signal'] = 1

# 1-8) ADX 시스템 (+DI vs -DI)
signals['DI_signal'] = 0
signals.loc[df['DMP_14'] > df['DMN_14'], 'DI_signal'] = -1
signals.loc[df['DMP_14'] < df['DMN_14'], 'DI_signal'] = 1


# === 2. 모멘텀(Momentum) 지표 ===

# 2-1) RSI
signals['RSI_signal'] = 0
signals.loc[df['RSI_14'] < 30, 'RSI_signal'] = -1
signals.loc[df['RSI_14'] > 70, 'RSI_signal'] = 1

# 2-2) Stochastic (%K/%D 교차 + 과매수·과매도)
signals['STOCH_signal'] = 0
signals.loc[
    (df['STOCHk_14_3_3'] > df['STOCHd_14_3_3']) & (df['STOCHk_14_3_3'] < 20),
    'STOCH_signal'
] = -1
signals.loc[
    (df['STOCHk_14_3_3'] < df['STOCHd_14_3_3']) & (df['STOCHk_14_3_3'] > 80),
    'STOCH_signal'
] = 1

# 2-3) ROC_10 (부호로만 판단)
signals['ROC_signal'] = 0
signals.loc[df['ROC_10'] < 0, 'ROC_signal'] = -1
signals.loc[df['ROC_10'] > 0, 'ROC_signal'] = 1

# 2-4) CCI
signals['CCI_signal'] = 0
signals.loc[df['CCI_20_0.015'] < -100, 'CCI_signal'] = -1
signals.loc[df['CCI_20_0.015'] > 100, 'CCI_signal'] = 1

# 2-5) MOM
signals['MOM_signal'] = 0
signals.loc[df['MOM_10'] < 0, 'MOM_signal'] = -1
signals.loc[df['MOM_10'] > 0, 'MOM_signal'] = 1

# 2-6) TRIX 교차
signals['TRIX_signal'] = 0
signals.loc[df['TRIX_15_9'] > df['TRIXs_15_9'], 'TRIX_signal'] = -1
signals.loc[df['TRIX_15_9'] < df['TRIXs_15_9'], 'TRIX_signal'] = 1

# 2-7) CMO
signals['CMO_signal'] = 0
signals.loc[df['CMO_14'] < -50, 'CMO_signal'] = -1
signals.loc[df['CMO_14'] > 50,  'CMO_signal'] = 1

# 2-8) TSI vs Signal
signals['TSI_signal'] = 0
signals.loc[df['TSI_13_25_13'] > df['TSIs_13_25_13'], 'TSI_signal'] = -1
signals.loc[df['TSI_13_25_13'] < df['TSIs_13_25_13'], 'TSI_signal'] = 1


# === 3. 변동성(Volatility) 지표 ===

# 3-1) ATR Ratio 변화
signals['ATRr_signal'] = 0
signals.loc[df['ATRr_14'] > df['ATRr_14'].shift(1), 'ATRr_signal'] = 1
signals.loc[df['ATRr_14'] < df['ATRr_14'].shift(1), 'ATRr_signal'] = -1

# 3-2) Bollinger Band %B
signals['BBP_signal'] = 0
signals.loc[df['BBP_20_2.0'] < 0.1, 'BBP_signal'] = -1
signals.loc[df['BBP_20_2.0'] > 0.9, 'BBP_signal'] = 1

# 3-3) Bollinger Band 폭 (추세 강도)
signals['BBB_signal'] = 0
signals.loc[df['BBB_20_2.0'].diff() > 0, 'BBB_signal'] = 1
signals.loc[df['BBB_20_2.0'].diff() < 0, 'BBB_signal'] = -1

# 3-4) Keltner Channel 돌파
signals['KC_signal'] = 0
signals.loc[df['Close'] > df['KCUe_20_2'], 'KC_signal'] = -1
signals.loc[df['Close'] < df['KCLe_20_2'], 'KC_signal'] = 1

# 3-5) Donchian Channel 돌파
signals['DC_signal'] = 0
signals.loc[df['Close'] > df['DCU_20_20'], 'DC_signal'] = -1
signals.loc[df['Close'] < df['DCL_20_20'], 'DC_signal'] = 1

# 3-6) NATR 변화
signals['NATR_signal'] = 0
signals.loc[df['NATR_14'] > df['NATR_14'].shift(1), 'NATR_signal'] = 1
signals.loc[df['NATR_14'] < df['NATR_14'].shift(1), 'NATR_signal'] = -1


# === 4. 거래량(Volume) 지표 ===

# 4-1) OBV 변화
signals['OBV_signal'] = 0
signals.loc[df['OBV'] > df['OBV'].shift(1), 'OBV_signal'] = -1
signals.loc[df['OBV'] < df['OBV'].shift(1), 'OBV_signal'] = 1

# 4-2) MFI
signals['MFI_signal'] = 0
signals.loc[df['MFI_14'] < 20, 'MFI_signal'] = -1
signals.loc[df['MFI_14'] > 80, 'MFI_signal'] = 1

# 4-3) AD (A/D Line) 변화
signals['AD_signal'] = 0
signals.loc[df['AD'] > df['AD'].shift(1), 'AD_signal'] = -1
signals.loc[df['AD'] < df['AD'].shift(1), 'AD_signal'] = 1

# 4-4) EFI 변화
signals['EFI_signal'] = 0
signals.loc[df['EFI_13'] > df['EFI_13'].shift(1), 'EFI_signal'] = -1
signals.loc[df['EFI_13'] < df['EFI_13'].shift(1), 'EFI_signal'] = 1

# 4-5) CMF
signals['CMF_signal'] = 0
signals.loc[df['CMF_20'] > 0, 'CMF_signal'] = -1
signals.loc[df['CMF_20'] < 0, 'CMF_signal'] = 1


# === 5. 기타/종합(Misc) 지표 ===

# 5-1) VWAP_D 돌파
signals['VWAP_signal'] = 0
signals.loc[df['Close'] > df['VWAP_D'], 'VWAP_signal'] = -1
signals.loc[df['Close'] < df['VWAP_D'], 'VWAP_signal'] = 1

# 5-2) DPO
signals['DPO_signal'] = 0
signals.loc[df['DPO_20'] > 0, 'DPO_signal'] = -1
signals.loc[df['DPO_20'] < 0, 'DPO_signal'] = 1

# 5-3) PGO
signals['PGO_signal'] = 0
signals.loc[df['PGO_14'] > df['PGO_14'].shift(1), 'PGO_signal'] = -1
signals.loc[df['PGO_14'] < df['PGO_14'].shift(1), 'PGO_signal'] = 1

# 5-4) BOP
signals['BOP_signal'] = 0
signals.loc[df['BOP'] > 0, 'BOP_signal'] = -1
signals.loc[df['BOP'] < 0, 'BOP_signal'] = 1


# 3) CSV 저장
signals.to_csv("All_TA_signals.csv", index=True)
print("✅ 저장 완료: All_TA_signals.csv")


✅ 저장 완료: All_TA_signals.csv
